In [ ]:
import json, os, requests, ipaddress, folium, math
from datetime import datetime
from collections import defaultdict

def filter_data(data, start_period, end_period):
    filtered_data = []

    for report_entry in data:
        for timestamp, report_data in report_entry.items():

            ts = datetime.strptime(timestamp, "%Y-%m-%d_%Hh%M")
            
            if start_period <= ts <= end_period:
                filtered_data.append({timestamp: report_data})

    return filtered_data

start = datetime(2025, 10, 31, 0, 0)
end = datetime(2025, 10, 31, 23, 59)

FILE_NAME = "reports.json"
with open(FILE_NAME, 'r') as f:
   d = json.load(f)

   data = filter_data(d, start, end)

freq = defaultdict(int)
delay_sum = defaultdict(float)
delay_count = defaultdict(int)
loss_sum = defaultdict(float)
loss_count = defaultdict(int)

points = {}
# TODO: ajouter les pertes dans le popup
for report_entry in data:
    for timestamp, report_data in report_entry.items():
        seen = set()
        for router in report_data["report"]["hubs"]:
            ip_address = router["host"]

            if ip_address == "???":
                continue

            if ip_address not in seen:
                freq[ip_address] += 1
                seen.add(ip_address)
            
            # collecter les statistiques de délai et de perte
            avg_delay = router.get("Avg")
            if isinstance(avg_delay, (int, float)):
                delay_sum[ip_address] += avg_delay
                delay_count[ip_address] += 1

            loss = router.get("Loss%")
            if isinstance(loss, (int, float)):
                loss_sum[ip_address] += loss
                loss_count[ip_address] += 1


# Get more in-depth info on those IP addresses
for ip in freq:
    url = f"https://ipinfo.io/{ip}"
    r = requests.get(url)
    if r.ok and "loc" in r.json():
        lat, lon = map(float, r.json()["loc"].split(","))
        points[ip] = {
            "lat": lat,
            "long": lon,
            "avg_delay": delay_sum[ip]/delay_count[ip],
            "avg_loss": loss_sum[ip]/loss_count[ip],
            "freq": freq[ip],
        }
   # Handle errors
    else:
        print(f"Error at IP {ip}: {r.status_code}")

with open("routers_pos.json", 'w', encoding="utf-8") as f:
    json.dump(points, f, ensure_ascii=False, indent=2)

first_key = next(iter(points))
# Création de la carte
m = folium.Map(location=[points[first_key]["lat"], points[first_key]["long"]], zoom_start=5)

def is_too_close(lat1, lon1, lat2, lon2, threshold=0.01):
    """verifier si deux points sont trop proches（threshold ecart de lon/lat 0.01° ≈ 1km）"""
    return math.sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2) < threshold

def adjust_position(lat, lon, existing_positions, step=0.01, max_iter=10):
    """si un point est trop proche d'un autre, le deplacer legerement"""
    for _ in range(max_iter):
        if all(not is_too_close(lat, lon, x, y) for x, y in existing_positions):
            return lat, lon
        # deplacement aleatoire pour eviter la collision
        lat += step * (0.5 - os.urandom(1)[0] / 255)
        lon += step * (0.5 - os.urandom(1)[0] / 255)
    return lat, lon  # si on n'arrive pas a eviter, retourner la position originale


# Ajout des points
drawn_ips = set()
existing_positions = []

# determiner la couleur en fonction de la perte
def loss_to_color(loss):
    if loss < 1:
        return "green"
    elif loss < 10:
        return "orange"
    else:
        return "red"

# tracer les points
for ip, info in points.items():
    lat, lon = info["lat"], info["long"]
    freq_val = info["freq"]
    delay_val = info["avg_delay"]
    loss_val = info["avg_loss"]

    # si l'IP a déjà été dessinée, sauter-la
    if ip in drawn_ips:
        continue
    drawn_ips.add(ip)

    # ajuster la position pour eviter les collisions des points proches
    lat, lon = adjust_position(lat, lon, existing_positions)
    existing_positions.append((lat, lon))

    # tracer les cercles
    folium.CircleMarker(
        location=[lat, lon],
        radius=3 + freq_val,
        color=loss_to_color(loss_val),
        fill=True,
        fill_color=loss_to_color(loss_val),
        fill_opacity=0.6,
        popup=f"{ip}<br>ttr_moyenne: {delay_val:.1f} ms<br>nb_occurrences: {freq_val} loss_moyenne: {loss_val:.1f}%<br>",
    ).add_to(m)

    # ajouter les labels
    folium.Marker(
        location=[lat, lon],
        icon=folium.DivIcon(
            html=f"<div style='font-size:8pt; color:black; text-align:center;'>"
                 f"<b>{ip}</b><br> {delay_val:.1f}ms<br> {freq_val}×</div> {loss_val:.1f}%<br>"
        ),
    ).add_to(m)

# Tracer les chemins de chaque rapport
for report_entry in data:
    for timestamp, content in report_entry.items():
        hubs = content["report"]["hubs"]
        route = [r["host"] for r in hubs if r["host"] != "???"]

        path_points = []
        for ip in route:
            if ip in points:
                lat, lon = points[ip]["lat"], points[ip]["long"]
                path_points.append((lat, lon))

        # creer une FeatureGroup pour chaque chemin
        if len(path_points) >= 2:
            fg = folium.FeatureGroup(name=f"Report {timestamp}")
            folium.PolyLine(
                locations=path_points,
                color="blue",
                weight=2,
                opacity=0.6,
                tooltip=f"chemin: {timestamp}"
            ).add_to(fg)
            fg.add_to(m)

# Ajouter les boutons de contrôle des couches
folium.LayerControl().add_to(m)


m.save("map_close.html")
print("✅ Carte enregistrée dans map_close.html")


/Users/gabu/Desktop/INSA/TC/Projet-MET/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Error at IP 192.168.1.1: 200
Error at IP 100.127.240.242: 200
Error at IP 172.16.50.33: 200
Error at IP 172.16.50.181: 200
Error at IP 172.16.50.190: 200
Error at IP 172.16.50.198: 200
✅ Carte enregistrée dans map_close.html
